## Austin Geovisualization 
### Set up environment

In [4]:
# set up environment 
import altair as alt
import pandas as pd
import geopandas as gpd
import OSMPythonTools as osmp # OpenStreetMap data 

### Import GeoJSON

#### Neighborhood Data

For geographic analyses, we need to import the geographic boundaries of Austin's neighborhoods. To do this, we can use GeoPandas' read file function: 

In [9]:
# neighborhood geojson import 
neighborhoods = gpd.read_file("austin.geojson")
neighborhoods.head()

,name,cartodb_id,created_at,updated_at,geometry
0,Blackland,1,2013-02-17T09:28:09.692000+00:00,2013-02-17T09:28:09.956000+00:00,"MULTIPOLYGON (((-97.72409 30.27926, -97.72514 ..."
1,Bouldin Creek,2,2013-02-17T09:28:09.692000+00:00,2013-02-17T09:28:09.956000+00:00,"MULTIPOLYGON (((-97.75962 30.24211, -97.76031 ..."
2,Brentwood,3,2013-02-17T09:28:09.692000+00:00,2013-02-17T09:28:09.956000+00:00,"MULTIPOLYGON (((-97.72354 30.33038, -97.72371 ..."
3,Cherrywood,4,2013-02-17T09:28:09.692000+00:00,2013-02-17T09:28:09.956000+00:00,"MULTIPOLYGON (((-97.70711 30.28920, -97.70700 ..."
4,Chestnut,5,2013-02-17T09:28:09.692000+00:00,2013-02-17T09:28:09.956000+00:00,"MULTIPOLYGON (((-97.71991 30.27379, -97.72010 ..."


#### School Data 

Now that we have Austin's geographic boundaries imported, we can also import Austin School District data: 

In [10]:
# schools geojson import 
schools = gpd.read_file("austin_schools.geojson")
schools.head()

,FID,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,...,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR,geometry
0,81891,480000811280,4800008,ROOSTER SPRINGS EL,48,1001 BELTERRA DR,AUSTIN,TX,78737,48,...,1,N,N,N,N,4825,48045,48025,2018-2019,POINT (-97.98292 30.19218)
1,81892,480000813086,4800008,SYCAMORE SPRINGS EL,48,14451 SAWYER RANCH RD,AUSTIN,TX,78737,48,...,1,N,N,N,N,4825,48045,48025,2018-2019,POINT (-98.00025 30.17858)
2,81893,480000813151,4800008,SYCAMORE SPRINGS MIDDLE,48,14451 SAWYER RANCH RD,AUSTIN,TX,78737,48,...,1,N,N,N,N,4825,48045,48025,2018-2019,POINT (-98.00025 30.17858)
3,81942,480001609410,4800016,AUSTIN CAN ACADEMY,48,2406 ROSEWOOD AVE,AUSTIN,TX,78702,48,...,1,N,N,N,N,4825,48046,48014,2018-2019,POINT (-97.70895 30.27222)
4,82016,480004408055,4800044,WAYSIDE EDEN PARK ACADEMY,48,6215 MANCHACA RD,AUSTIN,TX,78745,48,...,1,N,N,N,N,4821,48049,48014,2018-2019,POINT (-97.80292 30.20907)


### OpenStreetMap

The library we imorted, OSMPythonTools, allows us to query data from its databases - through OSMP, we are pulling a list of libraries in Austin. To do this, we can use OSMP's Nomatim function to query: 

In [11]:
from OSMPythonTools.nominatim import Nominatim

nominatim = Nominatim()
city = nominatim.query('Austin, Texas')
city.areaId()

3600113314

#### Library Data

After pulling the areaID, we can create a query using OSMP's overpassQueryBuilder: 

In [60]:
from OSMPythonTools.overpass import overpassQueryBuilder

# library query 
library_query = overpassQueryBuilder(
    area = city.areaId(), # areaID
    elementType = 'node', # for our data, we need nodes 
    selector = '"amenity"="library"', # we are pulling library data here 
    out = 'body', # body tells OSMP that we want the full data, not just the count 
    includeGeometry = True # includes geometry in our data 
) 

library_query

'area(3600113314)->.searchArea;(node["amenity"="library"](area.searchArea);); out body geom;'

Now that we've boilt our query, we can pass it to the Overpass function to return the query: 

In [61]:
from OSMPythonTools.overpass import Overpass

# query function
overpass = Overpass()

# library query results
libs = overpass.query(library_query)

With our library data query, we can now pull the data from one of our nodes as well as the geometry: 

In [62]:
# library data 
libs.nodes()[0].tags()

{'addr:state': 'TX',
 'amenity': 'library',
 'ele': '163',
 'gnis:county_name': 'Travis',
 'gnis:feature_id': '2360810',
 'gnis:import_uuid': '57871b70-0100-4405-bb30-88b2e001a944',
 'gnis:reviewed': 'no',
 'name': 'Texas State Law Library',
 'source': 'USGS Geonames'}

In [63]:
# library geometry 
libs.nodes()[0].geometry()

{"coordinates": [-97.741983, 30.276236], "type": "Point"}

Now that we have a better understanding of our data, we can pull a tuple of our data into a list using a list comprehension: 

In [64]:
# library list comprehension
libraries = [ (lib.tag("name"), lib.geometry() ) for lib in libs.nodes()]

Now that we have our data containing list, we can create a GeoDataFrame with it: 

In [65]:
# libraries geodataframe 
libraries = gpd.GeoDataFrame(libraries, columns = ['NAME', 'geometry'])
libraries.head()

,NAME,geometry
0,Texas State Law Library,POINT (-97.74198 30.27624)
1,Milwood Branch Austin City Library,POINT (-97.71617 30.42232)
2,Little Walnut Creek Branch Austin City Library,POINT (-97.69846 30.36327)
3,Southeast Austin Branch Austin City Library,POINT (-97.74193 30.18760)
4,Munday Library,POINT (-97.75750 30.23110)


#### Tree Data 

Now, let's do the same with the tree data: 

In [66]:
# tree query 
tree_query = overpassQueryBuilder(
    area = city.areaId(), # areaID
    elementType = 'node', # for our data, we need nodes 
    selector = '"natural"="tree"', # we are pulling library data here 
    out = 'body', # body tells OSMP that we want the full data, not just the count 
    includeGeometry = True # includes geometry in our data 
) 

# tree query results 
tree_data = overpass.query(tree_query)

# tree list comprehension
trees = [ (tree.id(), tree.geometry()) for tree in tree_data.nodes() ]

# trees geodataframe
trees = gpd.GeoDataFrame(trees, columns = ['name', 'geometry'])
trees.head()

,name,geometry
0,1079727457,POINT (-97.73878 30.27459)
1,1079727564,POINT (-97.73996 30.27296)
2,1079727579,POINT (-97.74045 30.27282)
3,1079727594,POINT (-97.74033 30.27275)
4,1079727602,POINT (-97.73910 30.27508)


Now that we have all our data, we can visualize it using Altair's Geoshape function: 

In [68]:
# basemap 
basemap = alt.Chart(neighborhoods).mark_geoshape(
    fill = 'lightgray', 
    stroke = 'darkgray'
).encode(tooltip = ['name']).properties(
    width = 600, 
    height = 600, 
    title = 'Austin School & Tree Locations'
)

# school map
school_map = alt.Chart(schools).mark_circle(
    opacity = 1,
    color = 'blue'
).encode(
    longitude = 'geometry.coordinates[0]:Q', 
    latitude = 'geometry.coordinates[1]:Q',
    tooltip = ['NAME', 'STREET', 'ZIP']
)

# library map 
library_map = alt.Chart(libraries).mark_circle(
    opacity = 1, 
    color = 'red'
).encode(
    longitude = 'geometry.coordinates[0]:Q', 
    latitude = 'geometry.coordinates[1]:Q',
    tooltip = ['NAME']
)

# tree map 
tree_map = alt.Chart(trees).mark_circle(
    size = 5, 
    opacity = 0.25, 
    color = 'green'
).encode(
    longitude = 'geometry.coordinates[0]:Q', 
    latitude = 'geometry.coordinates[1]:Q'
).properties(width=600, height=700)

# order here determines the vertical order 
basemap + tree_map + library_map + school_map 

alt.LayerChart(...)

Although OSM's data is quite extensive, it is by no means complete. In our Austin map, our school and library data seem to be quite reliable; our school data even contains schools outside of the geography of Austin neighborhoods. However, our tree data appears to be incomplete in a majority of the Austin geography. 

### Population Data

In our Austin geovisualization, we visualized one aspect of the data we had available, such as school, library, and tree locations. However, we can also visualize population data. For our next visualization, we are going to use population data from all over the world:

In [129]:
# geographic data 
geonames = pd.read_csv('https://www.geonames.org/countryInfoCSV', sep = '\t')
geonames.head()

,iso alpha2,iso alpha3,iso numeric,fips code,name,capital,areaInSqKm,population,continent,languages,currency,geonameId
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU,ca,EUR,3041565
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS,"ar-AE,fa,en,hi,ur",AED,290557
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS,"fa-AF,ps,uz-AF,tk",AFN,1149361
3,AG,ATG,28,AC,Antigua and Barbuda,St John's,443.0,96286,NaN,en-AG,XCD,3576396
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NaN,en-AI,XCD,3573511


From the data we imported, we only really need country name, isoalpha3, areaInSqKm, and population. Additionally, we need to set 'iso alpha3' as our index. 

In [130]:
# country geodataframe 
geonames = geonames[['name', 'iso alpha3', 'areaInSqKm', 'population']].set_index('iso alpha3')
geonames.head()

,name,areaInSqKm,population
iso alpha3,,,
AND,Andorra,468.0,77006
ARE,United Arab Emirates,82880.0,9630959
AFG,Afghanistan,647500.0,37172386
ATG,Antigua and Barbuda,443.0,96286
AIA,Anguilla,102.0,13254


Now that we have the geoname data we need, we need to also collect our geographic boundaries from our geojson file: 

In [131]:
# geojson file import 
polygons = gpd.read_file('countries.geo.json')

# drop name column 
polygons = polygons.drop(['name'], axis = 1)

# set index to three letter ID
polygons = polygons.set_index('id')

Our data contains a lot more detail than we actually need. In order to simplify the complexity of the data, we can use geopandas' simplify function:  

In [132]:
# simplify the complexity of shapes 
polygons.geometry = polygons.geometry.simplify(.1)
polygons.head()

,geometry
id,
AFG,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
AGO,"MULTIPOLYGON (((16.32653 -5.87747, 16.86019 -7..."
ALB,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
ARE,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
ARG,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."


Now that we have both of our dataframes created, we can join them on the three letter country code: 

In [133]:
# join dataframes 
countries = polygons.join(geonames, how='inner')
countries.head()

,geometry,name,areaInSqKm,population
AFG,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",Afghanistan,647500.0,37172386
AGO,"MULTIPOLYGON (((16.32653 -5.87747, 16.86019 -7...",Angola,1246700.0,30809762
ALB,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",Albania,28748.0,2866376
ARE,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",United Arab Emirates,82880.0,9630959
ARG,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",Argentina,2766890.0,44494502


Now that we've joined our dataframes, we can create new data from existing columns such as population density. To calculate population density, we need to take the poulation column and divide it by the areaInSqKm column: 

In [134]:
# create population density column 
countries['density'] = countries['population'] / countries['areaInSqKm']
countries.head()

,geometry,name,areaInSqKm,population,density
AFG,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",Afghanistan,647500.0,37172386,57.409090
AGO,"MULTIPOLYGON (((16.32653 -5.87747, 16.86019 -7...",Angola,1246700.0,30809762,24.713052
ALB,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",Albania,28748.0,2866376,99.706971
ARE,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",United Arab Emirates,82880.0,9630959,116.203656
ARG,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",Argentina,2766890.0,44494502,16.081052


We can only keep countries with a valid density value and turn our values into integers: 

In [136]:
# drop counties with invalid densities
countries = countries.drop(countries[countries.density <= 0].index, axis = 0)

# round and turn densities into integers
countries.density = countries.density.round(0).astype(int) 

In [137]:
countries.head()

,geometry,name,areaInSqKm,population,density
AFG,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",Afghanistan,647500.0,37172386,57
AGO,"MULTIPOLYGON (((16.32653 -5.87747, 16.86019 -7...",Angola,1246700.0,30809762,25
ALB,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",Albania,28748.0,2866376,100
ARE,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",United Arab Emirates,82880.0,9630959,116
ARG,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",Argentina,2766890.0,44494502,16


Now that we are done with data processing, we can visualize our countries data: 

In [143]:
alt.Chart(countries).mark_geoshape().encode(
    color = alt.Color('density', scale=alt.Scale(type='log', domain=[1,1000])), 
    tooltip = ['name', 'areaInSqKm', 'population', 'density']
).properties(width=600, height=600)

alt.Chart(...)